In [1]:
import os
import sys
import csv
import requests
import json
import operator
import time
import datetime
import platform
import itertools

In [2]:
def writeText(text, path, mode = 'w'):
    with open (path, mode) as textout:
        textout.write((text))
        
def writeJson(json, path, mode = 'w'):
    with open(path, mode) as file:
        file.write(json.dumps(json))
        
def writeCsv(listOut, outputFile):
    import csv
    with open (outputFile, "w", newline='', encoding = 'utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter = ",")
        for element in listOut:
            writer.writerow(element)

In [3]:
def getTxt(path):
    return open(path, 'r').read()

def getCsv(path, delim = ','):
    list_return = []
    with open (path, encoding = 'windows-1252') as file:
        csvreader = csv.reader(file, delimiter = delim)        
        for i, line in enumerate(csvreader):
            list_inner = []
            list_inner.append(line[0].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[1].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[2].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_return.append(list_inner)
    
    
    return list_return

In [4]:
if platform.system() == 'Windows':
    prep_data = getCsv('..\\output\\prep_out.csv')
else:
    prep_data = getCsv('../output/prep_out.csv')

In [5]:
print(len(prep_data))
print(prep_data[1141])

3573
[['agricultural', ' raw', ' materials', ' imports', ' merchandise', ' imports', ' countries'], ['agricultural', ' raw', ' materials', ' comprise', ' sitc', ' section', ' crude', ' materials', ' fuels', ' excluding', ' divisions', ' crude', ' fertilizers', ' minerals', ' excluding', ' coal', ' petroleum', ' precious', ' stones', ' metalliferous', ' ores', ' scrap'], ['']]


## Word2Vec

In [6]:
def getVecList(list_in):
    failcnt = 0
    failelemcnt = 0
    str_log = 'Started at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S') + '\n'
    print('Started at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'),'\n')
    # List of Lists as input needed
    adress = 'http://word2vec.ai.wu.ac.at/googlenews/model?word='
#     adress = 'http://word2vec.ai.wu.ac.at/twitter27_200/model?word='
    list_return = [[],[],[]]
    list_failed = []
    
    for i, elem in enumerate(list_in):
        inner_feat1, inner_feat2, inner_label = [], [], []
        str_1 = '-------------------------------\n' + 'Element ' + str(i)
        #print(str_1)
        str_log += str_1 + '\n'
# name ----------------------------------------
        for str_elem in elem[0]:
            word = str_elem.strip() 
            try:
                inner_feat1.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_feat1.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_2 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_2)
                    str_log += str_2 + '\n'
#                     inner_feat1.append([])
                    list_failed.append(word)
                    failcnt += 1
# desc ----------------------------------------
        for str_elem in elem[1]:
            word = str_elem.strip() 
            try:
                inner_feat2.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_feat2.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_3 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_3)
                    str_log += str_3 + '\n'
#                     inner_feat2.append([])
                    list_failed.append(word)
                    failcnt += 1
# label ----------------------------------------
        for str_elem in elem[2]:
            word = str_elem.strip() 
            try:
                inner_label.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_label.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_4 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_4)
                    str_log += str_4 + '\n'
#                     inner_label.append([])
                    list_failed.append(word)
                    failcnt += 1
        
        
        if (inner_feat1 and inner_feat2 and inner_label):
            list_return[0].append(inner_feat1)
            list_return[1].append(inner_feat2)
            list_return[2].append(inner_label)
        else:
            str_log += 'fail\nEmpty List can not be appended!\n'
            failelemcnt += 1
            
        
        
    print('Failed words: ', failcnt)
    print('Failed Elements: ', failelemcnt)
    str_log += '\nFinished at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    writeText(str_log, '../output/log.txt')    
    print('\nEnded at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')) 
    return list_return, list_failed

In [7]:
vect_list = getVecList(prep_data)

Started at:
2018-11-04 15:33:48 

Failed words:  4806
Failed Elements:  35

Ended at:
2018-11-04 15:52:04


In [8]:
feature1 = vect_list[0][0]
feature2 = vect_list[0][1]
labels = vect_list[0][2]
missing = vect_list[1]

In [9]:
len(feature1)

3538

In [10]:
if platform.system() == 'Windows':
    writeCsv(feature1, '..\\output\\feat1.csv')
    writeCsv(feature2, '..\\output\\feat2.csv')
    writeCsv(labels, '..\\output\\labels.csv')
else:
    writeCsv(feature1, '../output/feat1.csv')
    writeCsv(feature2, '../output/feat2.csv')
    writeCsv(labels, '../output/labels.csv')

In [11]:
print(feature1[0])
print('-----------------------------------')
print(feature2[0])
print('-----------------------------------')
print(labels[0])

[[-0.294921875, 0.150390625, -0.1484375, -0.0179443359375, -0.050048828125, -0.2109375, 0.1083984375, -0.12451171875, 0.076171875, 0.01055908203125, 0.08544921875, -0.039794921875, -0.052978515625, 0.01031494140625, -0.142578125, 0.08447265625, -0.057861328125, 0.1943359375, 0.09130859375, 0.1337890625, -0.0269775390625, -0.1513671875, 0.07568359375, -0.279296875, 0.036865234375, 0.1220703125, 0.0196533203125, 0.15625, 0.0400390625, 0.294921875, 0.054443359375, -0.359375, -0.20703125, -0.337890625, 0.058349609375, 0.003753662109375, -0.1953125, -0.053955078125, 0.287109375, 0.330078125, 0.1474609375, -0.0006561279296875, 0.031005859375, 0.2216796875, -0.236328125, -0.15625, 0.1962890625, 0.162109375, 0.0703125, -0.00494384765625, -0.033447265625, -0.05126953125, -0.2392578125, 0.421875, 0.08349609375, 0.03173828125, -0.1044921875, -0.054443359375, -0.023193359375, 0.00848388671875, -0.2421875, 0.1357421875, 0.0146484375, 0.0247802734375, 0.02294921875, -0.037109375, -0.03662109375, -0.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>



















